# Stay up to date on the latest trends with publicly available Google Trends data in BigQuery and the wiki API

This notebook provides some example code for running a BigQuery query to determine new rising search terms. We then use those terms as input for the wikipedia API to see if any of the terms are names of singers. You can use this code to develop a cloud function that runs each night to tell you if any new singers are trending!

Import libraries and authenticate into Google Cloud


In [1]:
import requests
import urllib
import time
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

ModuleNotFoundError: No module named 'google.colab'

In [2]:
%load_ext google.colab.data_table

ModuleNotFoundError: No module named 'google.colab'

Find new terms

In [ ]:
%%bigquery df
# select new search terms from the table
select distinct term from `bigquery-public-data.google_trends.top_rising_terms`
where refresh_date = DATE_SUB(CURRENT_DATE(), INTERVAL 1 DAY)

In [ ]:
df

,term
0,conor mcgregor injury
1,usa vs nigeria basketball
2,who won the mcgregor fight
3,england vs italy
4,england italy
5,ufc 264 results
6,crackstreams
7,virgin galactic launch
8,virgin galactic
9,richard branson


Create Helper Functions

In [ ]:
def add_url_params(url, param,term):
  # parse the url
  parsed = urllib.parse.urlparse(url)

  # convert query sting into dict
  parsed_get_args = dict(urllib.parse.parse_qsl(parsed.query))

  # add in the new param
  params = {param:term}
  parsed_get_args.update(params)

  # encode new query string and subsitute old query string with new one - return the new formatted ur;
  encoded_get_args =  urllib.parse.urlencode(parsed_get_args, doseq=True)
  newurl = urllib.parse.urlunparse([encoded_get_args if i == 4 else x for i,x in enumerate(parsed)])
  
  return newurl

In [ ]:
def is_singer(term):
  # to avoid hitting any limits
  time.sleep(2)

  # get the formatted search url for term and post
  search_url = add_url_params('https://en.wikipedia.org/w/api.php?action=opensearch&limit=1&namespace=0&format=json','search',term)
  s_resp = requests.post(search_url)
  
  # get the wiki page title from the response
  try:
    page_title = s_resp.json()[-1][0].split('/')[-1]
  except:
    return False
  
  # get the formatted url for getting content of page and post
  page_url = add_url_params('https://en.wikipedia.org/w/api.php?action=query&prop=extracts&exintro&format=json','titles',page_title)
  p_resp = requests.post(page_url)
  
  # extract first line from the HTML and return true if singer is listed
  try:
    pages = p_resp.json()['query']['pages']
    first_line = pages[list(pages.keys())[0]]['extract'].split('.')[0]
    print(first_line)
  except:
    return False
  
  return 'singer' in first_line.lower()

For each term, determine if they're a singer

In [ ]:
df['is_singer'] = df['term'].apply(lambda x: is_singer(x))

<!-- 
NewPP limit report
Parsed by mw2298
Cached time: 20210712135500
Cache expiry: 1814400
Reduced expiry: false
Complications: []
CPU time usage: 0
<p><b>Virgin Galactic</b> (VG) is a British-American spaceflight company that operates in the United States
<p class="mw-empty-elt">
</p>


<p class="mw-empty-elt">

</p>
<p><b>Sir Richard Charles Nicholas Branson</b> (born 18 July 1950) is an English business magnate, adventurer, investor, author, and commercial astronaut
<p><b>USA Basketball</b> (<b>USAB</b>) is a non-profit organization and the governing body for basketball in the United States
<p class="mw-empty-elt">
</p>

<p><b>Gregory McKarl Hardy</b> (born July 28, 1988) is an American mixed martial artist and former American football defensive end
<p><b>StreamCast Networks</b>, Inc
<p class="mw-empty-elt">


</p>
<p><b>Conor Anthony McGregor</b> (Irish: <i lang="ga">Conchúr Antóin Mac Gréagóir</i>; born 14 July 1988) is an Irish mixed martial artist, boxer, and businessman
<p cla

Get resulting dataframe - that shows each term and whether or not they're a singer

In [ ]:
df

,term,is_singer
0,conor mcgregor injury,False
1,usa vs nigeria basketball,False
2,who won the mcgregor fight,False
3,england vs italy,False
4,england italy,False
5,ufc 264 results,False
6,crackstreams,False
7,virgin galactic launch,False
8,virgin galactic,False
9,richard branson,False
